In [43]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
pd.set_option("display.max_rows", None)

In [97]:
class CrawlFuturePriceTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime('%Y/%m/%d')
        self.target_name = "台股期貨價格資訊"

    def crawl_main(self):
        para = '&queryStartDate=' + self.date_str + '&queryEndDate=' + self.date_str
        url = 'https://www.taifex.com.tw/cht/3/futDataDown?down_type=1&commodity_id=all&commodity_id2=' + para
        r = requests.post(url)
        lines = r.text.replace('\r', '').split('\n')
        lines = list(filter(lambda l:len(l.split(',')) >10, lines))
        if len(lines)<10:
            return None
        content = "\n".join(lines)
        df = pd.read_csv(StringIO(content))
        df=df.drop(columns='價差對單式委託成交量')
        df=df.reset_index()
        df.columns =lines[0].split(',')
        df=df.astype(str)
        df=df.drop(columns=['交易日期','漲跌價','歷史最高價','歷史最低價'])
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','contract_date','open','high',
                                                             'low','close','quote_change','turnover_vol',
                                                             'settlement_price','open_interest','best_bid','best_ask',
                                                            'trading_halt','trading_session','cross_contract_vol'])})


        to_numeric_col=list(df.columns)[2:]
        to_numeric_col.remove('trading_session')
        df[to_numeric_col]=df[to_numeric_col].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df['date']=self.date
        return df

    
z=CrawlFuturePriceTW(datetime.datetime(2018,1,3)).crawl_main()
z

,stock_id,contract_date,open,high,low,close,quote_change,turnover_vol,settlement_price,open_interest,best_bid,best_ask,trading_halt,trading_session,cross_contract_vol,date
0,XJF,201803,111.9200,111.9700,111.9000,111.9700,NaN,29.0,111.9700,394.0,111.9000,111.9300,NaN,一般,NaN,2018-01-03
1,XJF,201803,111.9500,111.9500,111.7300,111.8600,NaN,34.0,NaN,NaN,111.8200,111.9200,NaN,盤後,NaN,2018-01-03
2,XJF,201806,111.3000,111.3500,111.3000,111.3500,NaN,7.0,111.3500,186.0,111.2900,111.3200,NaN,一般,NaN,2018-01-03
3,XJF,201806,111.2500,111.2500,111.1000,111.2200,NaN,126.0,NaN,NaN,104.0000,111.3000,NaN,盤後,NaN,2018-01-03
4,XJF,201809,NaN,NaN,NaN,NaN,NaN,0.0,110.8600,5.0,103.4000,118.8000,NaN,一般,NaN,2018-01-03
5,XJF,201809,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,103.4000,118.8000,NaN,盤後,NaN,2018-01-03
6,XJF,201812,NaN,NaN,NaN,NaN,NaN,0.0,109.9700,0.0,102.5000,117.8000,NaN,一般,NaN,2018-01-03
7,XJF,201812,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,102.5000,117.8000,NaN,盤後,NaN,2018-01-03
8,XIF,201801,12501.0000,12535.0000,12490.0000,12516.0000,NaN,174.0,12516.0000,169.0,12503.0000,12533.0000,NaN,一般,NaN,2018-01-03
9,XIF,201802,12451.0000,12502.0000,12445.0000,12487.0000,NaN,22.0,12487.0000,35.0,12486.0000,12501.0000,NaN,一般,NaN,2018-01-03
